In [219]:
# TODO: address copy/slice issues
# for nickname handling, the order needs to be:
# Get nicknames, merge into SSa names that are common
# collapse into root name, name
# train on nickname
import pandas as pd
import requests
from wordfreq import word_frequency
import nltk
nltk.download('words')
from nltk.corpus import words
from tqdm import tqdm
tqdm.pandas()
import pync
import csv
from zipfile import ZipFile

[nltk_data] Downloading package words to
[nltk_data]     /Users/wraphaelson/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [220]:
# some helper functions
def in_words(word):
    return word in words.words()


def lookup_word_freq(word):
    try:
        freq = word_frequency(word, lang="en", wordlist='best', minimum=0.0)
        return freq
    except Exception as e:
        pass

In [221]:
# get nicknames data. we'll end up analyzing a dataset of nicknames + names 
# from the SSA as long as > 100 people actually got that name in 2019
url = "https://github.com/carltonnorthern/nickname-and-diminutive-names-lookup/blob/master/names.csv?raw=true"
nicknames = pd.read_csv(url, header=None, names=(["name"]+list(range(20))))

In [222]:
nicknames

,name,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,aaron,erin,ronnie,ron,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,abbigail,nabby,abby,gail,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abednego,bedney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abel,ebbie,ab,abe,eb,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abiel,ab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,zach,zachariah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1033,zachariah,zachy,zach,zeke,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1034,zebedee,zeb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1035,zedediah,dyer,zed,diah,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
# # melt, dropdupe, drop nas
# nicknames = nicknames.melt()
# nicknames = nicknames[['value']]
# nicknames = nicknames.drop_duplicates()
# nicknames = nicknames.dropna()
# nicknames.columns = ["name"]
# nicknames

In [224]:
# get ssa names data for frequencies
url = "https://www.ssa.gov/oact/babynames/names.zip"

with requests.get(url) as response:

    with open("names.zip", "wb") as temp_file:
        temp_file.write(response.content)

In [225]:
# compute name frequencies
# This list will hold all our data. We initialize it with the header row.
data_list = [["year", "name", "gender", "count"]]

# We first read the zip file using a zipfile.ZipFile object.
with ZipFile("names.zip") as temp_zip:

    # Then we read the file list.
    for file_name in temp_zip.namelist():

        # We will only process .txt files.
        if ".txt" in file_name:

            # Now we read the current file from the zip file.
            with temp_zip.open(file_name) as temp_file:

                # The file is opened as binary, we decode it using utf-8 so it can be manipulated as a string.
                for line in temp_file.read().decode("utf-8").splitlines():

                    # We prepare our desired data fields and add them to the data list.
                    line_chunks = line.split(",")
                    year = file_name[3:7]
                    name = line_chunks[0].lower()
                    gender = line_chunks[1]
                    count = line_chunks[2]

                    data_list.append([year, name, gender, count])

# We save the data list into a csv file.
csv.writer(open("data.csv", "w", newline="", encoding="utf-8")).writerows(data_list)

In [215]:
name_freqs = pd.read_csv("data.csv")
# just 2019
name_freqs = name_freqs[name_freqs['year'] == 2019]
# just names and counts
name_freqs = name_freqs[["name", "count"]]
name_freqs.columns = ["root_name", "count"]
# sum across genders
name_freqs = name_freqs.groupby(['root_name']).sum()
name_freqs.reset_index()


,root_name,count
0,aaban,6
1,aabha,5
2,aabriella,6
3,aadam,20
4,aadarsh,15
...,...,...
29346,zyrie,15
29347,zyrielle,5
29348,zyron,13
29349,zyrus,9


In [214]:
name_freqs

,count
root_name,
aaban,6
aabha,5
aabriella,6
aadam,20
aadarsh,15
...,...
zyrie,15
zyrielle,5
zyron,13


In [216]:
names = name_freqs.merge(nicknames, how="left", left_on="root_name", right_on="name")

In [218]:
names.to_csv("merge_test.csv")

In [211]:
names = names.melt(id_vars=["root_name", "count"])

KeyError: "The following 'id_vars' are not present in the DataFrame: ['root_name']"

In [201]:
names

,name,count,variable,value
0,NaN,6,0,NaN
1,NaN,5,0,NaN
2,NaN,6,0,NaN
3,NaN,20,0,NaN
4,NaN,15,0,NaN
...,...,...,...,...
587035,NaN,15,19,NaN
587036,NaN,5,19,NaN
587037,NaN,13,19,NaN
587038,NaN,9,19,NaN


In [182]:
names.to_csv("testing_melt.csv")
# we're almost there, we need to make sure the root names are in the final column as well. 
# just copy the root name to a new column in the initial manipulation and then we can melt it onto itself
# however then we'll miss SSA names that arent in the set, so just fillna with the root name in the final dataset

In [ ]:
names = name_freqs[name_freqs['count'] > 99]
names.reset_index(inplace=True)
names.columns = ["name", "count"]
names

In [145]:
# union nicknames and SSA names, drop drop dupes
full_names_dataset = names.append(nicknames)
full_names_dataset.drop_duplicates(keep='first')
full_names_dataset

,count,name
0,249.0,aadhya
1,154.0,aadya
2,3641.0,aaliyah
3,100.0,aamir
4,141.0,aanya
...,...,...
1848,NaN,maggy
1849,NaN,daisy
1850,NaN,gretta
1851,NaN,peggie


In [147]:
# determine if the name is a word in the nltk corpus and subset where True
full_names_dataset['word'] = full_names_dataset['name'].progress_apply(in_words)
full_names_dataset = full_names_dataset[full_names_dataset['word'] == True]
pync.notify('Done', title="Python")
full_names_dataset

100%|██████████| 5382/5382 [06:33<00:00, 13.67it/s]


,count,name,word
19,9117.0,abigail,True
25,1684.0,ace,True
106,2136.0,alan,True
108,1083.0,alani,True
111,259.0,alannah,True
...,...,...,...
1833,NaN,ginger,True
1837,NaN,dobbin,True
1842,NaN,robin,True
1846,NaN,mollie,True


In [148]:
# lookup word frequency in Exquisite Corpus
full_names_dataset['freq'] = full_names_dataset['name'].apply(lookup_word_freq)
full_names_dataset

<ipython-input-148-d7342836d2a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_names_dataset['freq'] = full_names_dataset['name'].apply(lookup_word_freq)


,count,name,word,freq
19,9117.0,abigail,True,2.190000e-06
25,1684.0,ace,True,1.200000e-05
106,2136.0,alan,True,2.750000e-05
108,1083.0,alani,True,6.310000e-08
111,259.0,alannah,True,8.710000e-08
...,...,...,...,...
1833,NaN,ginger,True,7.760000e-06
1837,NaN,dobbin,True,1.860000e-07
1842,NaN,robin,True,2.000000e-05
1846,NaN,mollie,True,5.890000e-07


In [149]:
# sort, clean up, and label
full_names_dataset = full_names_dataset.sort_values(by="freq", ascending=False)
full_names_dataset = full_names_dataset[["name", "count", "freq"]]
full_names_dataset.columns = ["name", "n_people_2019", "frequency_per_billion_words_in_corpus"]
full_names_dataset.head(50)

,name,n_people_2019,frequency_per_billion_words_in_corpus
1015,will,NaN,0.002880
3386,will,216.0,0.002880
1805,one,NaN,0.002880
1397,than,NaN,0.001320
1469,see,NaN,0.001260
2497,may,133.0,0.000977
700,may,NaN,0.000977
1574,day,NaN,0.000851
1193,life,NaN,0.000741
2287,love,138.0,0.000589
